# 环境设置&库与model下载

In [1]:
%nvidia-smi    #GPU设置
import os
HOME = os.getcwd()
print("HOME:", HOME)
%cd {HOME}
%git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
%git checkout -q 57535c5a79791cb76e36fdb64975271354f10251#文件的哈希码
%pip install -q -e .
#Grounding-DINO下载
%cd {HOME}
import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
#segment_anything下载
%pip uninstall -y supervision
%pip install -q supervision==0.6.0
import supervision as sv
print(sv.__version__)
#supervision
%pip install -q roboflow
#roboflow

#load model
%cd {HOME}
%mkdir -p {HOME}/weights
%cd {HOME}/weights
%wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))
%cd {HOME}
%mkdir -p {HOME}/weights
%cd {HOME}/weights
%wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
%cd {HOME}/GroundingDINO
from groundingdino.util.inference import Model
grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)
SAM_ENCODER_VERSION = "vit_h"
from segment_anything import sam_model_registry, SamPredictor
sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)

UsageError: Line magic function `%nvidia-smi` not found.


In [ ]:
#文件加载&放置
%cd {HOME}
%mkdir {HOME}/data
%cd {HOME}/data
SOURCE_IMAGE_PATH = f"{HOME}/data_1/out_2.jpg"#文件地址
#CLASSES = ['fresh apple', 'rooted apple']
CLASSES=['lemon','orange','pear','strawberry','kiwi fruit']
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25
#参数设置

import cv2
import supervision as sv

# load image
image = cv2.imread(SOURCE_IMAGE_PATH)

# detect objects
detections = grounding_dino_model.predict_with_classes(
    image=image,
    classes=enhance_class_name(class_names=CLASSES),
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD
)

# annotate image with detections
box_annotator = sv.BoxAnnotator()
labels = [
    f"{CLASSES[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _
    in detections]
annotated_frame = box_annotator.annotate(scene=image.copy(), detections=detections, labels=labels)

%matplotlib inline
sv.plot_image(annotated_frame, (16, 16))

In [ ]:
#边缘分割：
import numpy as np
from segment_anything import SamPredictor


def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)
import cv2

#边缘覆盖
detections.mask = segment(
    sam_predictor=sam_predictor,
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
    xyxy=detections.xyxy
)

# 标识检测文件
box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()
labels = [
    f"{CLASSES[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _
    in detections]
annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

%matplotlib inline
path = f"{HOME}/data_1/"+'out.jpg'
sv.plot_image(annotated_image, (16, 16))
cv2.imwrite(path,annotated_image)
#生成标识并导出

In [ ]:
#黑白分割文件
import math
grid_size_dimension = math.ceil(math.sqrt(len(detections.mask)))
titles = [
    CLASSES[class_id]
    for class_id
    in detections.class_id
]
sv.plot_images_grid(
    images=detections.mask,
    titles=titles,
    grid_size=(grid_size_dimension, grid_size_dimension),
    size=(16, 16)
)
digit=0
for image in detections:
  title=titles[digit]+str(digit)+'.jpg'
  path = f"{HOME}/data_1/"+title+str(digit)
  cv2.imwrite(path,image)
  digit+=1


In [ ]:
#多文件同class生成
import os

IMAGES_DIRECTORY = os.path.join(HOME, 'data_1')
IMAGES_EXTENSIONS = ['jpg', 'jpeg', 'png']

CLASSES=['lemon','orange','pear','strawberry','kiwi fruit']
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

import cv2
from tqdm.notebook import tqdm

images = {}
annotations = {}

image_paths = sv.list_files_with_extensions(
    directory=IMAGES_DIRECTORY,
    extensions=IMAGES_EXTENSIONS)

for image_path in tqdm(image_paths):
    image_name = image_path.name
    image_path = str(image_path)
    image = cv2.imread(image_path)

    detections = grounding_dino_model.predict_with_classes(
        image=image,
        classes=enhance_class_name(class_names=CLASSES),
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    detections = detections[detections.class_id != None]
    detections.mask = segment(
        sam_predictor=sam_predictor,
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
        xyxy=detections.xyxy
    )
    images[image_name] = image
    annotations[image_name] = detections

plot_images = []
plot_titles = []

box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()

for image_name, detections in annotations.items():
    image = images[image_name]
    plot_images.append(image)
    plot_titles.append(image_name)

    labels = [
        f"{CLASSES[class_id]} {confidence:0.2f}"
        for _, _, confidence, class_id, _
        in detections]
    annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)
    annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
    plot_images.append(annotated_image)
    title = " ".join(set([
        CLASSES[class_id]
        for class_id
        in detections.class_id
    ]))
    plot_titles.append(title)

sv.plot_images_grid(
    images=plot_images,
    titles=plot_titles,
    grid_size=(len(annotations), 2),
    size=(2 * 4, len(annotations) * 4)
)